In [ ]:
import pandas as pd
import warnings
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scint
import math
import time
from lmfit import minimize, Parameters, Parameter, report_fit
warnings.filterwarnings("ignore")

In [ ]:
def bee_eq(y, t, w1,w2, external_temperature, truth, peri, treat):
    hive_temperature = y
    u1 = [0]*len(hive_temperature)
    u2 = [0]*len(hive_temperature)
    for i in range(len(hive_temperature)):
        if external_temperature[i] >= 0:
            u1[i] = 1
        else:
            u2[i] = -1
    if treat == 0:
        dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature + external_temperature)
    else:
        dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature) + external_temperature + peri
    return dydt

def run_bee_eq(t, a, w1, w2, ext_temp, truth, peri, treat):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, ext_temp, truth, peri, treat), col_deriv = True, rtol = 1e-3, atol = 1e-3) # 'w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, ext_temp, truth, peri, treat):
    #x0 = ps['a0'].value
    a = [10]*len(ext_temp)
    w1 = ps['w1'].value
    w2 = ps['w2'].value
    model = run_bee_eq(ts, a, w1, w2, ext_temp - theta_ideal,truth - theta_ideal, peri-theta_ideal, treat)
    return (model - truth).ravel()

In [ ]:
def main(*argv):
    filename = argv[0]
    temp_data = argv[1]
    ext_data = argv[2]
    peri_data = argv[3]
    type_ = argv[4]
    # set parameters incluing bounds
    params = Parameters()
    #params.add('a0', value=float(data['Temp_265'].values[0]), min=20, max=50)
    params.add('w1', value= 1.0, min=0, max=100)
    params.add('w2', value=1.0, min=0, max=100)


    data = pd.read_csv(filename)

    d = pd.DataFrame(data).groupby(data['Date'])
    sig_min, param_min, error = [], [], []
    time_ = []
    start = time.time()
    for i,j in d:
        print(i)
    
        t_max = len(j[ext_data])-1
        t = np.linspace(0,t_max, num = t_max+1)
        yy_param = np.linspace(min(j[temp_data]), max(j[temp_data]), num = 50)
        
        if type_ == 'Y':
            zz = j[peri_data].values
            treat = j['Treatment'].values[0]
        else:
            zz = j[temp_data].values
            treat = 0
        # fit model and find predicted values
        param = []
        error = []
        sig = []
        for k in yy_param:
            result = minimize(residual, params, args=(t, j[ext_temp].values-k, j[temp_data].values-k, zz, treat), method='leastsq')
            for name,param_ in result.params.items():
                if name == 'w':
                    param.append([k,param_.value])
            sig.append(j[temp_data] + result.residual.reshape(j[temp_data].shape))
            error.append(math.sqrt(np.mean((result.residual.reshape(j[temp_data].shape)**2))))
        end = time.time()
        error_arg = np.argmin(error)
        error_min.append(error[error_arg])
        sig_min.append(sig[error_arg])
        param_min.append(param[error_arg])
        
        time_.append(np.round((end - start)/60,4))

In [ ]:
if __name__ == "__main__":
    hive_number = input('Enter Hive Number: ')
    hive_type = input('Is it treated? Y or N: ')
    filename = 'C_'+hive_number+'.csv'
    temp_data = 'Temp_'+ hive_number
    ext_data = 'Temp_Box'
    peri_data = 'Temp_Easy_'+ hive_number
    main(filename, temp_data, ext_data, peri_data, hive_type)